(시도 반복)
1. 지역검색 클릭
2. 시도 클릭
3. 전체 클릭
4. 테이블 가져오기

(데이터 모델)
이름, 주소, 서비스, 타입, 위치_및_시설, 위도, 경도

# 스타벅스 매장 크롤링하기 - 샘플코드

In [217]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import openpyxl
import time
import re # 매장코드, 위도, 경도 추출

wait = WebDriverWait(driver, 10)

## 드라이버 세팅

In [188]:
driver = webdriver.Chrome('C:/chromedriver.exe')

## 메인 페이지 접속

In [189]:
driver.get('https://www.starbucks.co.kr/store/store_map.do')

## 지역검색버튼 클릭

In [190]:
driver.find_element_by_css_selector('#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a').click()

## 시도버튼 클릭

In [191]:
sido = driver.find_elements_by_css_selector('#container > div > form > fieldset > div > section > article.find_store_cont > article > article > div.loca_step1 > div.loca_step1_cont > ul > li > a')

In [192]:
sido[0].click()

## 전체버튼 클릭

In [193]:
# 세종은 전체버튼이 없음
try:
    driver.find_element_by_css_selector('#mCSB_2_container > ul > li:nth-child(1) > a').click()
except:
    pass

## 이름, 주소, 코드, 위도, 경도 추출

In [194]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

Stores = soup.select('#mCSB_3_container > ul > li')
print(len(Stores))

530


In [277]:
Stores[0]

<li class="quickResultLstCon" data-code="3627" data-hlytag="null" data-index="0" data-lat="36.494878" data-long="127.255384" data-name="세종다정DT" data-storecd="1505" style="background:#fff"> <strong>세종다정DT  <img alt="" class="setStoreFavBtn mCS_img_loaded" data-my_siren_order_store_yn="N" data-name="세종다정DT" data-store="1505" data-yn="N" src="//image.istarbucks.co.kr/common/img/store/icon_fav_off.png"/></strong> <p class="result_details">세종특별자치시 다정북로 219 (다정동)<br/>1522-3232</p> <i class="pin_generalDT">리저브 매장 2번</i></li>

In [196]:
# 이름 추출
name = Stores[0].find('strong').get_text()
name = name.strip()
name

'역삼아레나빌딩'

In [283]:
# 주소 추출
address = Stores[0].find('p', attrs={'class':'result_details'}).get_text()

# 전화번호 제거
address = re.sub('\d+-\d+', '', address)
address

'세종특별자치시 다정북로 219 (다정동)'

In [197]:
# '매장코드 추출' -> 나중에 js함수 구동할 때 필요
code = re.search(r'data-code="\d+', str(Stores[0])).group()
print(code)
code = code[11:]
code

data-code="3762


'3762'

In [198]:
# 위도 추출
lat = re.search(r'data-lat="\d+\.+\d+', str(Stores[0])).group()
print(lat)
lat = lat[10:]
lat

data-lat="37.501087


'37.501087'

In [199]:
# 경도 추출
long = re.search(r'data-long="\d+\.+\d+', str(Stores[0])).group()
print(long)
long = long[11:]
long

data-long="127.043069


'127.043069'

## 매장코드별 js함수 구동

In [285]:
driver.execute_script("getStoreDetail('%s')" % code)

## 정보 가져오기

In [306]:
html_2 = driver.page_source
soup_2 = BeautifulSoup(html_2, 'html.parser')

### 타입, 서비스, 위치_및_시설 가져오기

In [286]:
# 서비스 딕셔너리 생성 -> 전역으로 설정할 것
type_Dict = {'01': '스타벅스 리저브',
            '02' : '커뮤니티 스토어',
            '03' : '드라이브 스루'}

serv_Dict = {'04' : '주차',
            '23' : '블론드',
            '22' : '피지오',
            '18' : '나이트로 콜드 브루 커피',
            '20' : '현금없는 매장',
            '05' : '외화 결제',
            '24' : '식약처 위생등급제 인증'}

wich_Dict = {'07' : '공항내',
            '09' : '해안가',
            '08' : '대학가',
            '12' : '터미널/기차역',
            '11' : '리조트',
            '13' : '병원',
            '10' : '입점',
            '14' : '지하철 인접',
            '19' : '장애인 편의시설',
            '21' : '공기청정기'}

In [307]:
# 타입, 서비스, 위치_및_시설 아이콘 가져오기
icons = soup_2.select('div.shopArea_infoWrap > dl > dd > span > img')
icons

[<img class="mCS_img_loaded" src="/common/img/store/store_icon/icon04.png"/>,
 <img class="mCS_img_loaded" src="/common/img/store/store_icon/icon20.png"/>,
 <img class="mCS_img_loaded" src="/common/img/store/store_icon/icon22.png"/>,
 <img class="mCS_img_loaded" src="//image.istarbucks.co.kr/upload/common/img/icon/icon23.png"/>,
 <img class="mCS_img_loaded" src="/common/img/store/store_icon/icon19.png"/>]

In [291]:
# 파일명 추출
icons = re.findall(r'icon\d+\.\w+', str(icons))
icons

# 번호만 재추출
icon_number = re.findall(r'\d{2}', str(icons))
icon_number

['04', '20', '22', '23', '19']

In [292]:
매장유형_temp = []
서비스_temp = []
위치_및_시설_temp = []

for k in icon_number:
    # 매장유형이 적혀져있는 경우
    if '01' or '02' or '03' in icon_number:
        if k in type_Dict.keys():
            매장유형_temp.append(type_Dict[k])
        elif k in serv_Dict.keys():
            서비스_temp.append(serv_Dict[k])
        else:
            위치_및_시설_temp.append(wich_Dict[k])
        
    # 매장유형이 적혀져있지 않은 경우
    else:
        if k in serv_Dict.keys():
            서비스_temp.append(serv_Dict[k])
        else:
            위치_및_시설_temp.append(wich_Dict[k])

In [293]:
', '.join(매장유형_temp)

''

In [295]:
', '.join(서비스_temp)

'주차, 현금없는 매장, 피지오, 블론드'

In [298]:
', '.join(위치_및_시설_temp)

'장애인 편의시설'

In [335]:
try:
    driver.implicitly_wait(10)
    driver.find_element_by_css_selector('div.shopArea_pop01.isStoreBizViewWrap > p').click()
except:
    try:
        driver.implicitly_wait(10)
        driver.find_element_by_xpath('//*[@id="container"]/div/p').click()
    except:
        pass

# 스타벅스 매장 크롤링하기 - 메인코드

In [346]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import openpyxl
import time
import re # 매장코드, 위도, 경도 추출

city = input('시/도를 입력하세요 : ')

""" 전역변수 설정 """
city_Dict = {'서울':0,
             '경기':1,
             '광주':2,
             '대구':3,
             '대전':4,
             '부산':5,
             '울산':6,
             '인천':7,
             '강원':8,
             '경남':9,
             '경북':10,
             '전남':11,
             '전북':12,
             '충남':13,
             '충북':14,
             '제주':15,
             '세종':16}

이름 = []
매장코드 = []
위도 = []
경도 = []
매장유형 = []
서비스 = []
위치_및_시설 = []
주소 = []

# 딕셔너리 생성 -> 전역으로 설정할 것
type_Dict = {'01': '스타벅스 리저브',
            '02' : '커뮤니티 스토어',
            '03' : '드라이브 스루'}

serv_Dict = {'04' : '주차',
            '23' : '블론드',
            '22' : '피지오',
            '18' : '나이트로 콜드 브루 커피',
            '20' : '현금없는 매장',
            '05' : '외화 결제',
            '24' : '식약처 위생등급제 인증'}

wich_Dict = {'07' : '공항내',
            '09' : '해안가',
            '08' : '대학가',
            '12' : '터미널/기차역',
            '11' : '리조트',
            '13' : '병원',
            '10' : '입점',
            '14' : '지하철 인접',
            '19' : '장애인 편의시설',
            '21' : '공기청정기'}

count = 0

wait = WebDriverWait(driver, 10)


""" 기본 세팅 """
driver = webdriver.Chrome('C:/chromedriver.exe')

# 메인 페이지 점속
driver.get('https://www.starbucks.co.kr/store/store_map.do')

# 지역검색버튼 클릭
driver.implicitly_wait(10)
driver.find_element_by_css_selector('#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a').click()

# 시도버튼 클릭
sido = driver.find_elements_by_css_selector('#container > div > form > fieldset > div > section > article.find_store_cont > article > article > div.loca_step1 > div.loca_step1_cont > ul > li > a')
sido[city_Dict[city]].click()

# 전체버튼 클릭
# 세종은 전체버튼이 없음
try:
    driver.find_element_by_css_selector('#mCSB_2_container > ul > li:nth-child(1) > a').click()
except:
    pass
time.sleep(5)

""" 매장정보 불러오기 """
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

Stores = soup.select('#mCSB_3_container > ul > li')

""" 매장별 반복 """
for i in range(len(Stores)):
    """ 이름, 주소, 코드, 위도, 경도 불러오기 """
    # 이름 추출
    name = Stores[i].find('strong').get_text()
    name = name.strip()
    이름.append(name)
    
    # 주소 추출
    address = Stores[i].find('p', attrs={'class':'result_details'}).get_text()

    # 전화번호 제거
    address = re.sub('\d+-\d+', '', address)
    주소.append(address)

    # '매장코드 추출' -> 나중에 js함수 구동할 때 필요
    code = re.search(r'data-code="\d+', str(Stores[i])).group()
    code = code[11:]
    매장코드.append(code)

    # 위도 추출
    lat = re.search(r'data-lat="\d+\.+\d+', str(Stores[i])).group()
    lat = lat[10:]
    위도.append(lat)

    # 경도 추출
    long = re.search(r'data-long="\d+\.+\d+', str(Stores[i])).group()
    long = long[11:]
    경도.append(long)

    """ 매장코드별 js함수 구동 """
    time.sleep(1)
    driver.execute_script("getStoreDetail('%s')" % code)
    
    """ 정보들 가져오기 """
    time.sleep(2)
    html_2 = driver.page_source
    time.sleep(2)
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    """ 타입, 서비스, 위치_및_시설 가져오기 """
    # 아이콘 가져오기
    icons = soup_2.select('div.shopArea_infoWrap > dl > dd > span > img')
    
    # 파일명 추출
    icons = re.findall(r'icon\d+\.\w+', str(icons))
    
    # 번호만 재추출
    icon_number = re.findall(r'\d{2}', str(icons))

    # 임시 리스트에 담았다 원래 변수에 한꺼번에 저장
    매장유형_temp = []
    서비스_temp = []
    위치_및_시설_temp = []
    
    # 아이콘 추출
    for k in icon_number:
        # 매장유형이 적혀져있는 경우
        if '01' or '02' or '03' in icon_number:
            if k in type_Dict.keys():
                매장유형_temp.append(type_Dict[k])
            elif k in serv_Dict.keys():
                서비스_temp.append(serv_Dict[k])
            else:
                위치_및_시설_temp.append(wich_Dict[k])

        # 매장유형이 적혀져있지 않은 경우
        else:
            if k in serv_Dict.keys():
                서비스_temp.append(serv_Dict[k])
            else:
                위치_및_시설_temp.append(wich_Dict[k])

    매장유형.append(', '.join(매장유형_temp))
    서비스.append(', '.join(서비스_temp))
    위치_및_시설.append(', '.join(위치_및_시설_temp))
    
    # js함수 닫기 -> 일부에서 적용 안됨
    # js함수 닫기 -> 일부에서 적용 안됨
    try:
        driver.implicitly_wait(10)
        driver.find_element_by_css_selector('div.shopArea_pop01.isStoreBizViewWrap > p').click()
    except:
        try:
            driver.implicitly_wait(10)
            driver.find_element_by_xpath('//*[@id="container"]/div/p').click()
        except:
            pass
    
    print('[%d] : %s is Crawled' % (count+1, name))
    count += 1

""" df로 만들기 """
result = {'code':매장코드,
          'name':이름,
          'address':주소,
          'type':매장유형,
          'service':서비스,
          'loc&fac':위치_및_시설,
          'lat':위도,
          'long':경도}

df = pd.DataFrame(result,
                 columns=['code', 'name', 'address', 'type', 'service', 'loc&fac', 'lat', 'long'])

df.to_excel('%s_Stores.xlsx' % city, encoding='cp949')

print('\nCrawling is done')

시/도를 입력하세요 : 서울
[1] : 역삼아레나빌딩 is Crawled
[2] : 논현역사거리 is Crawled
[3] : 신사역성일빌딩 is Crawled
[4] : 국기원사거리 is Crawled
[5] : 스탈릿대치R is Crawled
[6] : 봉은사역 is Crawled
[7] : 압구정윤성빌딩 is Crawled
[8] : 코엑스별마당 is Crawled
[9] : 삼성역섬유센터R is Crawled
[10] : 압구정R is Crawled
[11] : 수서역R is Crawled
[12] : 양재강남빌딩R is Crawled
[13] : 선릉동신빌딩R is Crawled
[14] : 봉은사로선정릉 is Crawled
[15] : 강남오거리 is Crawled
[16] : 스타필드코엑스몰R is Crawled
[17] : 강남구청정문 is Crawled
[18] : 도곡공원 is Crawled
[19] : 강남R is Crawled
[20] : 대치은마사거리 is Crawled
[21] : 청담영동대로 is Crawled
[22] : 압구정 is Crawled
[23] : 신사가로수 is Crawled
[24] : 청담스타R is Crawled
[25] : 강남우성 is Crawled
[26] : 학동사거리 is Crawled
[27] : 대치사거리 is Crawled
[28] : 포스코사거리 is Crawled
[29] : 을지병원사거리 is Crawled
[30] : 신사역 is Crawled
[31] : 도곡역 is Crawled
[32] : 서울세관사거리 is Crawled
[33] : 청담사거리 is Crawled
[34] : 한티역 is Crawled
[35] : 역삼초교사거리 is Crawled
[36] : 코엑스몰 is Crawled
[37] : 가로수길 is Crawled
[38] : 역삼대로 is Crawled
[39] : 역삼포스코 is Crawled
[40] : 강남대로 is Crawled
[41] : 청담 is Crawl

[336] : 여의도역R is Crawled
[337] : 선유동이레빌딩 is Crawled
[338] : 동여의도 is Crawled
[339] : 여의도ABL타워 is Crawled
[340] : 영등포신길DT is Crawled
[341] : 문래역 is Crawled
[342] : 당산 is Crawled
[343] : 여의도한양증권 is Crawled
[344] : 여의도일신 is Crawled
[345] : 신세계타임스퀘어B2 is Crawled
[346] : 하이테크시티 is Crawled
[347] : 여의도 is Crawled
[348] : 롯데캐슬여의도 is Crawled
[349] : 문래동 is Crawled
[350] : 신세계타임스퀘어1F is Crawled
[351] : 여의도호성 is Crawled
[352] : 타임스퀘어B2 is Crawled
[353] : 영등포본동 is Crawled
[354] : 타임스퀘어2F is Crawled
[355] : 여의도의사당 is Crawled
[356] : 여의도IFC(1F) is Crawled
[357] : 당산역사거리 is Crawled
[358] : 불광역 is Crawled
[359] : 구파발사거리 is Crawled
[360] : 연신내역사거리 is Crawled
[361] : 연신내 is Crawled
[362] : 구파발역 is Crawled
[363] : 은평구청입구 is Crawled
[364] : 은평이마트 is Crawled
[365] : 구산역 is Crawled
[366] : 연합뉴스 is Crawled
[367] : 혜화역 is Crawled
[368] : 경희궁로 is Crawled
[369] : 강북삼성병원 is Crawled
[370] : 종로평창동 is Crawled
[371] : 광화문우체국 is Crawled
[372] : 더종로R is Crawled
[373] : 독립문역 is Crawled
[374] : 정부서울청사R is Crawled
[375] :